In [ ]:
!pip install sme

In [ ]:
import sme
from matplotlib import pyplot as plt

####  Importing a model
- to load an existing model: `sme.open_sbml_file('model_filename.xml')`
- to load a built-in example model: `sme.open_example_model()`

In [ ]:
my_model = sme.open_example_model()

####  Getting help
- to see the type of an object: `type(object)`
- to print a one line description of an object: `repr(object)`
- to print a multi-line description of an object: `print(object)`
- to get help on an object, its methods and properties: `help(object)`

In [ ]:
type(my_model)

In [ ]:
repr(my_model)

In [ ]:
print(my_model)

In [ ]:
help(my_model)

####  Editing model parameters
- model parameters can be changed by assigning new values to them

In [ ]:
print(my_model.name)

In [ ]:
my_model.name = 'New model name!'

In [ ]:
print(my_model.name)

####  Model contents
- the compartments in a model are stored as a dictionary, with the compartment names as keys
- a compartment can be looked up using its name: `model.compartments['compartment name']`
- or the compartments can be iterated over: `for name, compartment in model.compartments.items():`
- the species in a compartment can be accessed in the same way

In [ ]:
type(my_model.compartments)

In [ ]:
print(my_model.compartments)

In [ ]:
my_compartment = my_model.compartments['Cell']
print(my_compartment)

In [ ]:
for name, compartment in my_model.compartments.items():
    print(name)
    print(compartment)

In [ ]:
compartment_img = my_model.compartment_image()

plt.title("Compartment Geometry")
plt.xlabel("x")
plt.ylabel("y")
plt.imshow(compartment_img, interpolation='none')
plt.show()

####  Simulation
- models can be simulated by specifying the total simulation time, and the interval between images
- after simulation, a list of available time points is provided
- along with a concentration image for each time point

In [ ]:
my_model.simulate(simulation_time=250.0, image_interval=50.0)

In [ ]:
time_points = my_model.simulation_time_points()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=len(time_points), figsize=(18, 12))
for index, (ax, time_point) in enumerate(zip(axs, time_points)):
    ax.imshow(my_model.concentration_image(index))
    ax.set_title(f"t = {time_point}")
plt.show()